# Compare momentum and spin angles
This is from an MDC2 MC production. 

In [1]:
using PyCall, Plots, ProgressMeter

## Find data

Let's find some data in `/pnfs`. 

In [2]:
const filesDir = "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784"
files = readlines(`find $filesDir -name 'gm2*.30?*'`)  # Pick a subset of the files
println("There are $(length(files)) files")

There are 11 files


Let's look at one of the files to see what data products are in it. We can use the `product_sizes_dumper` command from _art_. 

In [3]:
files[1]

"/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.302.root"

In [4]:
;product_sizes_dumper $(files[1]) \| grep ::

         723258918      0.661 gm2truth::TrajectoryArtRecords_artg4__mdc2.
         244324304      0.223 gm2truth::TrackingActionArtRecords_artg4__mdc2.
          63004078      0.058 gm2truth::RingArtRecords_artg4_Ring_mdc2.
          37009378      0.034 gm2truth::LookupArtRecords_artg4_calorimeter_mdc2.
           7901880      0.007 gm2truth::RingTrackingPlaneArtRecords_artg4_RingTrackingPlanes_mdc2.
           6670241      0.006 gm2truth::GhostDetectorArtRecords_artg4_trackerdummyplane_mdc2.
           3557883      0.003 gm2truth::GhostFiberHarpArtRecords_artg4_ghostFiberHarp_mdc2.
           2155258      0.002 gm2truth::IBMSTruthArtRecords_artg4_ibms_mdc2.
           1792840      0.002 art::RNGsnapshots_randomsaver__mdc2.
           1110349      0.001 gm2truth::XtalArtRecords_artg4_calorimeter_mdc2.
           1077636      0.001 gm2truth::T0ArtRecords_artg4_t0_mdc2.
            772505      0.001 gm2truth::IBMSFiberArtRecords_artg4_ibms_mdc2.
            524512      0.000 gm2truth::Ph

Looks like we want the `RingTrackingPlaneArtRecords` with module label of `artg4` and instance label of `RingTrackingPlanes`. Here are details:

* Data product definition at [RingTrackingPlanesArtRecord.hh](https://redmine.fnal.gov/redmine/projects/gm2dataproducts/repository/revisions/v7_06_05/entry/mc/ring/RingTrackingPlaneArtRecord.hh) ... note it refers to [BasicArtRecord.hh](https://redmine.fnal.gov/redmine/projects/gm2dataproducts/repository/revisions/v7_06_05/entry/mc/basic/BasicArtRecord.hh)
* It gets made with [RingTrackingPlanes_service.hh](https://redmine.fnal.gov/redmine/projects/gm2ringsim/repository/revisions/v7_06_05/entry/ring/RingTrackingPlanes_service.hh) and [RingTrackingPlanes_service.cc](https://redmine.fnal.gov/redmine/projects/gm2ringsim/repository/revisions/v7_06_05/entry/ring/RingTrackingPlanes_service.cc)

We can look at the FHICL parameters with (note the `grep -A ...` means to show more lines after the match)

In [5]:
;config_dumper --services $(files[1]) \| grep  -A 4 RingTrackingPlanes

RingTrackingPlanes: {
   nplanes: 4
   phi0: -1
   verbosity: 0
}


Relevant parts of the construction code are ...

```cxx
phi0_{pset.get<double>("phi0", 0) * deg},
```

and

```cxx
for (int nplane=0; nplane!=nplanes_; ++nplane)
{
   // ...
   // Calculate downstream angle from 12 o'clock
   // in global gm2ringsim coordinates
   double angle = phi0_ + CLHEP::twopi * nplane / nplanes_;
   auto rot = new G4RotationMatrix();
   rot->rotateZ(angle);
   // ...
 ```
 
 Not sure why we start at -1 degree. 
 
 Also, note that 
 ```cxx
 typedef std::vector<RingTrackingPlaneArtRecord> RingTrackingPlaneArtRecordCollection;
 ```

## Setup Gallery and PyRoot

In [6]:
ROOT = pyimport("ROOT")

# We need two functions to set up PyRoot
#   Note that ROOT[:gROOT][:ProcessLine](...) in Julia is ROOT.gROOT.ProcessLine(...) in python  - wrap this!
function read_header(h)
    ROOT[:gROOT][:ProcessLine]("""#include "$h" """)
end

function provide_get_valid_handle(klass)
    ROOT[:gROOT][:ProcessLine]("template gallery::ValidHandle<$klass> gallery::Event::getValidHandle<$klass>(art::InputTag const&) const;")
end

# Here's the one header we need
read_header("gallery/ValidHandle.h")

0

We need to set up a template for the data we're going to extract

In [7]:
provide_get_valid_handle("std::vector<gm2truth::RingTrackingPlaneArtRecord>")

0

## Prepare to run

Define the input tag

In [8]:
tag = ROOT[:art][:InputTag]("artg4:RingTrackingPlanes")

PyObject <ROOT.art::InputTag object at 0xcb8c770>

Load the files vector

In [9]:
mdc2_1033 = ROOT[:vector](ROOT[:string])()
for aFile ∈ files
    mdc2_1033[:push_back](aFile)
end

mdc2_1033[:size]()

11

Get the event iterator

In [10]:
ev = ROOT[:gallery][:Event](mdc2_1033)

Successfully opened file /pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.302.root


PyObject <ROOT.gallery::Event object at 0x72de720>

## Find an event with lots of ring hits

Define a function to retrieve a valid handle to the data we want to extract

In [11]:
get_rtp = ev[:getValidHandle](ROOT[:vector](ROOT[:gm2truth][:RingTrackingPlaneArtRecord]))

PyObject <ROOT.MethodProxy object at 0x7fb29595cfd0>

In [92]:
# Let's just return one with a lot of turns
function loop(ev, maxNum)
    nSeen = 0
    while ! ev[:atEnd]() && nSeen < maxNum
    
        rtpH = get_rtp(tag)
        
        if rtpH[:size]() > 2000
            return rtpH
        end
    
        nSeen += 1
        ev[:next]()
    end
end

loop (generic function with 1 method)

Loop over the events and find one with lots of hits

In [93]:
rtpH = loop(ev, 5000)

PyObject <ROOT.gallery::ValidHandle<vector<gm2truth::RingTrackingPlaneArtRecord> > object at 0xf5502b0>

In [94]:
@assert typeof(rtpH) != Void

How many hits did we get?

In [95]:
rtpH[:size]()

3045

In [96]:
py"$ev.eventEntry()"

294

Let's see where the hits are

In [97]:
rtpv = rtpH[:product]()

PyObject <ROOT.vector<gm2truth::RingTrackingPlaneArtRecord> object at 0xda99d00>

In [98]:
2+2

4

In [99]:
function makeCols(rtpv)
    
    hitx = Float64[] ; hity = Float64[]; hitz = Float64[]; hittime = Float64[]
    parentid = Int[]
    
    for h in rtpv
        b = h[:basicInfo]
        pos = b[:pos]
        
        push!(hitx, pos[:x]())
        push!(hity, pos[:y]())
        push!(hitz, pos[:z]())
        
        push!(parentid, b[:parentid])
        push!(hittime, b[:time])
    end
    
    return hitx, hity, hitz, parentid, hittime
    
end

makeCols (generic function with 1 method)

In [100]:
function makeCols2(rtpv)
    
    hitx = Float64[] ; hity = Float64[]; hitz = Float64[]; hittime = Float64[]
    parentid = Int[]
    
    @time for h in rtpv
        push!(hitx, py"$h.basicInfo.pos.x()")
        push!(hity, py"$h.basicInfo.pos.y()")
        push!(hitz, py"$h.basicInfo.pos.z()")
        push!(parentid, py"$h.basicInfo.parentid")
        push!(hittime, py"$h.basicInfo.time")

    end
    
    return hitx, hity, hitz, parentid, hittime
    
end

makeCols2 (generic function with 1 method)

In [101]:
@time hitx, hity, hitz, parentid, hittime = makeCols(rtpv) ; 

  2.618145 seconds (193.93 k allocations: 7.713 MiB, 1.23% gc time)


In [102]:
@time hitx, hity, hitz, parentid, hittime = makeCols2(rtpv) ; 

  0.946765 seconds (85.32 k allocations: 1.709 MiB)
  0.997147 seconds (92.50 k allocations: 2.112 MiB)


In [103]:
function makeCols3(rtpv)
    
    hitx = py"[h.basicInfo.pos.x() for h in $rtpv]"
    hity = py"[h.basicInfo.pos.y() for h in $rtpv]"
    hitz = py"[h.basicInfo.pos.z() for h in $rtpv]"
    hittime = py"[h.basicInfo.time for h in $rtpv]"
    parentid = py"[h.basicInfo.parentid for h in $rtpv]"

    return hitx, hity, hitz, hittime, parentid
end

makeCols3 (generic function with 1 method)

In [104]:
@time hitx, hity, hitz, hittime, parentid = makeCols3(rtpv)

  0.265662 seconds (30.07 k allocations: 813.558 KiB)


([7182.42, 124.109, -7100.45, -124.061, 7128.44, 124.15, -7101.13, -123.981, 7127.01, 124.226  …  -124.276, 7106.3, 123.871, -7120.79, -124.317, 7110.49, 123.841, -7116.31, -124.334, 7115.13], [-12.0132, 5.1441, 17.5241, 25.5454, 27.2004, 22.0393, 11.4067, -2.04544, -15.0118, -24.2157  …  -11.5703, -21.7873, -26.5648, -24.7515, -16.7581, -4.58976, 8.69135, 19.8175, 26.0249, 25.7253], [-125.168, 7115.92, 123.839, -7113.17, -124.327, 7118.27, 123.851, -7108.59, -124.302, 7122.65  …  -7125.5, -123.941, 7102.26, 124.194, -7127.88, -124.014, 7100.57, 124.116, -7128.84, -124.095], [22.6031, 60.106, 97.3631, 134.618, 171.963, 209.324, 246.598, 283.841, 321.166, 358.537  …  1.13243e5, 1.1328e5, 1.13317e5, 1.13355e5, 113392.0, 1.13429e5, 1.13467e5, 1.13504e5, 1.13541e5, 1.13579e5], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [106]:
plotly()

Plots.PlotlyBackend()

In [107]:
scatter(hitz, hitx)

There are the four ring planes! Let's look in 3D

In [46]:
scatter3d(hitz, hitx, hity)
xlabel!("z") ; ylabel!("x")

How does $y$ oscillate?

In [47]:
plot(hittime, hity, marker=true, width=0.5, markersize=1, xlabel="ns", ylabel="y", xlims=(0, 11_000), labels="")

... More to do ...